In [5]:
import pandas as pd
from matplotlib import pyplot as plt
import mysql.connector
import folium

In [6]:
# AWS INSTANCE에서 데이터 가져오기

yespa_db = mysql.connector.connect(
    host = 'database-1.c3ieyo8m4v9o.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'eda_team',
    database = 'eda_project',
    password = '123456'
)

cur = yespa_db.cursor(buffered=True, dictionary=True)

cur.execute('SELECT * FROM GIHU_metro_coor')
result = cur.fetchall()

for data in result:
    print(data)
    
yespa_db.close()

{'역명': '창동', '호선': '1호선', '위도': 37.653181, '경도': 127.047663}
{'역명': '월계', '호선': '1호선', '위도': 37.633114, '경도': 127.058835}
{'역명': '외대앞', '호선': '1호선', '위도': 37.596157, '경도': 127.063572}
{'역명': '대방', '호선': '1호선', '위도': 37.513392, '경도': 126.926407}
{'역명': '신길', '호선': '1호선', '위도': 37.517018, '경도': 126.917743}
{'역명': '구일', '호선': '1호선', '위도': 37.496273, '경도': 126.869599}
{'역명': '온수', '호선': '1호선', '위도': 37.492267, '경도': 126.823399}
{'역명': '소요산', '호선': '1호선', '위도': 37.5653, '경도': 127.0339}
{'역명': '도봉산', '호선': '1호선', '위도': 37.689603, '경도': 127.046347}
{'역명': '방학', '호선': '1호선', '위도': 37.66796, '경도': 127.04456}
{'역명': '녹천', '호선': '1호선', '위도': 37.644599, '경도': 127.051406}
{'역명': '석계', '호선': '1호선', '위도': 37.615206, '경도': 127.065594}
{'역명': '신이문', '호선': '1호선', '위도': 37.60274474, '경도': 127.067556}
{'역명': '회기', '호선': '1호선', '위도': 37.589802, '경도': 127.057936}
{'역명': '남영', '호선': '1호선', '위도': 37.540597, '경도': 126.971171}
{'역명': '용산', '호선': '1호선', '위도': 37.52989, '경도': 126.964775}
{'역명': '노량진', '호선': '1호선'

In [7]:
df=pd.DataFrame(result)
df

,역명,호선,위도,경도
0,창동,1호선,37.653181,127.047663
1,월계,1호선,37.633114,127.058835
2,외대앞,1호선,37.596157,127.063572
3,대방,1호선,37.513392,126.926407
4,신길,1호선,37.517018,126.917743
...,...,...,...,...
401,한티,수인분당,37.496208,127.052883
402,도곡,수인분당,37.490914,127.055393
403,대모산입구,수인분당,37.491422,127.072740
404,수서,수인분당,37.487472,127.101422


In [8]:
numlist=[]

for 호선 in sorted(df['호선'].unique()):
    count = (df['호선'] == 호선).sum()
    print(f"{호선}은 {count}개")
    numlist.append(count)

1호선은 35개
2호선은 50개
3호선은 34개
4호선은 26개
5호선은 52개
6호선은 39개
7호선은 42개
8호선은 18개
9호선은 38개
경의중앙은 22개
경춘은 7개
공항철도은 6개
수인분당은 13개
신림은 11개
우이신설선은 13개


In [9]:
seoul_metro_1to9_colors =['darkblue', 'lightgreen', 'orange', 'lightblue', 'darkpurple',
                             'darkred', 'darkgreen', 'pink', 'lightgray']
subway_lines = {
    "우이신설선": "lightgreen",
    "신림": "blue",
    "공항철도": "darkblue",
    "경의중앙": "lightblue",
    "경춘": "green",
    "수인분당": "lightorange",
}


In [10]:
seoul_metro_colors_dict = {}

for i in range(1, 10):
    seoul_metro_colors_dict[f"{i}호선"] = seoul_metro_1to9_colors[i-1]

seoul_metro_colors_dict

{'1호선': 'darkblue',
 '2호선': 'lightgreen',
 '3호선': 'orange',
 '4호선': 'lightblue',
 '5호선': 'darkpurple',
 '6호선': 'darkred',
 '7호선': 'darkgreen',
 '8호선': 'pink',
 '9호선': 'lightgray'}

In [11]:
seoul_metro_colors_dict.update(subway_lines)
seoul_metro_colors_dict

{'1호선': 'darkblue',
 '2호선': 'lightgreen',
 '3호선': 'orange',
 '4호선': 'lightblue',
 '5호선': 'darkpurple',
 '6호선': 'darkred',
 '7호선': 'darkgreen',
 '8호선': 'pink',
 '9호선': 'lightgray',
 '우이신설선': 'lightgreen',
 '신림': 'blue',
 '공항철도': 'darkblue',
 '경의중앙': 'lightblue',
 '경춘': 'green',
 '수인분당': 'lightorange'}

In [12]:
df_colors = pd.DataFrame(list(seoul_metro_colors_dict.items()), columns=['호선', '색상'])
df_colors

,호선,색상
0,1호선,darkblue
1,2호선,lightgreen
2,3호선,orange
3,4호선,lightblue
4,5호선,darkpurple
5,6호선,darkred
6,7호선,darkgreen
7,8호선,pink
8,9호선,lightgray
9,우이신설선,lightgreen


In [13]:
import folium

m = folium.Map(location=[37.58, 127.0], zoom_start=10)

for i in range(0,sum(numlist[:1])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[0],
                                    icon='1',
                                    prefix='fa'),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:1]),sum(numlist[:2])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[1], 
                                    icon='2',
                                    prefix='fa'),
                  popup =df['역명'][i]
                  ).add_to(m)   
for i in range(sum(numlist[:2]),sum(numlist[:3])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[2], 
                                    icon='3',
                                    prefix='fa'),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:3]),sum(numlist[:4])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[3], 
                                    icon='4',
                                    prefix='fa'),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:4]),sum(numlist[:5])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[4], 
                                    icon='5',
                                    prefix='fa'),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:5]),sum(numlist[:6])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[5], 
                                    icon='6',
                                    prefix='fa'),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:6]),sum(numlist[:7])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[6], 
                                    icon='7',
                                    prefix='fa'),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:7]),sum(numlist[:8])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[7], 
                                    icon='8',
                                    prefix='fa'),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:8]),sum(numlist[:9])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = seoul_metro_1to9_colors[8], 
                                    icon='9',
                                    prefix='fa'),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:9]),sum(numlist[:10])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = subway_lines[df['호선'][i]]),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:10]),sum(numlist[:11])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = subway_lines[df['호선'][i]]),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:11]),sum(numlist[:12])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = subway_lines[df['호선'][i]]),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:12]),sum(numlist[:13])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = subway_lines[df['호선'][i]]),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:13]),sum(numlist[:14])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = subway_lines[df['호선'][i]]),
                  popup =df['역명'][i]
                  ).add_to(m)
for i in range(sum(numlist[:14]),sum(numlist[:15])):
    folium.Marker([df['위도'][i],df['경도'][i]], 
                  icon =folium.Icon(color = subway_lines[df['호선'][i]]),
                  popup =df['역명'][i]
                  ).add_to(m) 
m

/tmp/ipykernel_22490/1808997321.py:95: UserWarning: color argument of Icon should be one of: {'darkpurple', 'pink', 'darkred', 'gray', 'darkblue', 'black', 'purple', 'darkgreen', 'lightblue', 'lightgray', 'green', 'orange', 'blue', 'cadetblue', 'white', 'beige', 'lightred', 'lightgreen', 'red'}.
  icon =folium.Icon(color = subway_lines[df['호선'][i]]),


In [14]:
import folium

m = folium.Map(location=[37.58, 127.0], zoom_start=10)

for i in range(len(numlist)):
    for j in range(sum(numlist[:i]), sum(numlist[:i+1])):
        icon_color = seoul_metro_1to9_colors[i] if i < 9 else subway_lines[df['호선'][j]]
        icon_number = str(i+1) if i < 9 else None
        folium.Marker([df['위도'][j], df['경도'][j]], 
                      icon=folium.Icon(color=icon_color, icon=icon_number, prefix='fa'),
                      popup=df['역명'][j]).add_to(m)

m

/tmp/ipykernel_22490/2616010896.py:10: UserWarning: color argument of Icon should be one of: {'darkpurple', 'pink', 'darkred', 'gray', 'darkblue', 'black', 'purple', 'darkgreen', 'lightblue', 'lightgray', 'green', 'orange', 'blue', 'cadetblue', 'white', 'beige', 'lightred', 'lightgreen', 'red'}.
  icon=folium.Icon(color=icon_color, icon=icon_number, prefix='fa'),


In [16]:
import mysql.connector

remote = mysql.connector.connect(
    host = 'database-1.c3ieyo8m4v9o.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'eda_team',
    database = 'eda_project',
    password = '123456'
)

cursor = remote.cursor(buffered=True)
cursor.execute("CREATE TABLE GIHU_metro_colors(호선 VARCHAR(32), 색상 VARCHAR(16))")
sql = "INSERT INTO GIHU_metro_colors VALUES(%s,%s)"

for i, row in df_colors.iterrows():
    cursor.execute(sql, tuple(row))
    print(tuple(row))
    remote.commit()

('1호선', 'darkblue')
('2호선', 'lightgreen')
('3호선', 'orange')
('4호선', 'lightblue')
('5호선', 'darkpurple')
('6호선', 'darkred')
('7호선', 'darkgreen')
('8호선', 'pink')
('9호선', 'lightgray')
('우이신설선', 'lightgreen')
('신림', 'blue')
('공항철도', 'darkblue')
('경의중앙', 'lightblue')
('경춘', 'green')
('수인분당', 'lightorange')
